# CZ3006 Lab4
By Benjamin Tan

The goal of this assignment is to analyse the given data_2.csv traffic log data, and to generate the following information:

 1. Top 5 talkers
 2. Top 5 listeners
 3. Top 5 applications
 4. Total estimated traffic (in bytes)
 5. Proportion of TCP and UDP packets

Furthermore we will also generate:
 6. Top 5 communication pairs

Let us start by first importing the relevant packages:

In [1]:
import pandas as pd
import numpy as np
import requests #useful for retrieving organization names from ip addresses
import math

Next is to retrieve our data. We will also setup data retrieval requests from the website IP-API for this, since they have organization names for ip addresses.

In [2]:
data_cols = ['type', 'sflow_agent_address', 'inputPort', 'outputPort', 'src_MAC', 'dst_MAC', 'ethernet_type',
'in_vlan', 'out_vlan', 'src_IP', 'dst_IP', 'IP_protocol', 'ip_tos', 'ip_ttl', 'src_port', 'dst_port',
'tcp_flags', 'packet_size', 'IP_size', 'sampling_rate']
ip_csv_url = "http://ip-api.com/csv"
df = pd.read_csv(
    "Data_2.csv",
    index_col=False,
    names=data_cols
)
df.head(10)

,type,sflow_agent_address,inputPort,outputPort,src_MAC,dst_MAC,ethernet_type,in_vlan,out_vlan,src_IP,dst_IP,IP_protocol,ip_tos,ip_ttl,src_port,dst_port,tcp_flags,packet_size,IP_size,sampling_rate
0,FLOW,203.30.38.251,193,130,0031466b23cf,00239cd087c1,0x0800,919.0,919,74.125.10.59,137.132.38.238,17,0x00,63,443.0,62758,0x00,1396,1378,2048
1,FLOW,203.30.38.251,193,130,0031466b23cf,00239cd087c1,0x0800,919.0,919,173.194.22.215,137.132.228.29,6,0x00,63,443.0,39740,0x10,1438,1420,2048
2,FLOW,203.30.38.251,199,131,544b8cf9a7df,001cb0c88e40,0x0800,600.0,43,193.62.193.9,202.130.56.153,6,0x00,246,80.0,35794,0x18,272,250,2048
3,FLOW,203.30.38.251,193,131,0031466b23cf,001cb0c88e40,0x0800,919.0,43,74.125.130.132,192.122.131.33,6,0x00,52,443.0,5287,0x10,1418,1400,2048
4,FLOW,203.30.38.251,129,193,00135f21bc80,0031466b23cf,0x0800,11.0,919,155.69.160.69,74.125.68.139,17,0x60,59,48361.0,443,0x00,89,67,2048
5,FLOW,203.30.38.251,135,129,002688cd5fc7,00135f21bc80,0x0800,919.0,11,54.230.159.64,155.69.191.254,6,0x00,251,443.0,16657,0x10,1418,1400,2048
6,FLOW,203.30.38.251,129,193,00135f21bc80,0031466b23cf,0x0800,11.0,919,155.69.253.4,74.125.10.39,6,0x00,63,51374.0,443,0x10,74,52,2048
7,FLOW,203.30.38.251,129,193,00135f21bc80,0031466b23cf,0x0800,11.0,919,155.69.160.73,64.233.188.128,6,0x00,56,23153.0,443,0x10,1422,1400,2048
8,FLOW,203.30.38.251,200,130,80711fc76001,00239cd087c1,0x0800,280.0,919,129.99.230.54,137.132.22.74,6,0x00,53,48975.0,15000,0x10,1442,1420,2048
9,FLOW,203.30.38.251,200,129,80711fc76001,00135f21bc80,0x0800,280.0,11,210.32.0.65,155.69.16.255,6,0x00,41,80.0,56552,0x10,1522,1500,2048


Now to generate information.

# 1. Top 5 Talkers

In [3]:
talkers = df['src_IP'].value_counts().to_frame()
talkers.columns = ['No. of packets']
talkers = talkers.head(5)
talkers

,No. of packets
13.107.4.50,5960
130.14.250.7,4034
155.69.160.38,3866
171.67.77.19,2656
155.69.199.255,2587


We want the organization names as well, so:

In [4]:
print(f'{"Organization":50} {"IP_Address":20} {"No of packets":8}')
print('-' * 80)
for index, row in talkers.iterrows():
    org = requests.get(f'{ip_csv_url}/{index}?fields=org').text
    org = org.strip()
    print(f'{org:50}{index:20}{row["No. of packets"]:>8}')

Organization                                       IP_Address           No of packets
--------------------------------------------------------------------------------
Microsoft Corporation                             13.107.4.50             5960
National Library of Medicine                      130.14.250.7            4034
Nanyang Technological University                  155.69.160.38           3866
Stanford University                               171.67.77.19            2656
Nanyang Technological University                  155.69.199.255          2587


# 2. Top 5 Listeners
Same thing as above.

In [5]:
listeners = df['dst_IP'].value_counts().to_frame()
listeners.columns = ['No. of packets']
listeners = listeners.head(5)
print(f'{"Organization":50} {"IP_Address":20} {"No of packets":8}')
print('-' * 80)
for index, row in listeners.iterrows():
    org = requests.get(f'{ip_csv_url}/{index}?fields=org').text
    org = org.strip()
    print(f'{org:50}{index:20}{row["No. of packets"]:>8}')

Organization                                       IP_Address           No of packets
--------------------------------------------------------------------------------
National University of Singapore                  137.132.228.33          5908
A*STAR                                            192.122.131.36          4662
Nusgp                                             202.51.247.133          4288
National University of Singapore                  137.132.228.29          4022
A*STAR                                            103.37.198.100          3741


Since we only care about the number of packets recieved by an IP address, and not an organization, we will stop here even though both NUS and ASTAR repeat in the Top 5 listeners.

# 3. Top 5 Applications
Since we don't have a data file to convert port numbers into service names, we have to hardcode the application names manually.

In [6]:
applications = df['dst_port'].value_counts().to_frame()
applications.columns = ['No. of packets']
applications = applications.head(5)
applications

,No. of packets
443,43208
80,11018
50930,2450
15000,2103
8160,1354


In [7]:
applicationname = ("HTTPS", "HTTP", "Dynamic Port", "Registered Port", "Registered Port")
print(f'{"Service":40} {"Port No.":15} {"No. of packets":12}')
print('-' * 70)
appdigit = 0
for index, row in applications.iterrows():
    print(f'{applicationname[appdigit]:40}{index:15}{row["No. of packets"]:>12}')
    appdigit+=1

Service                                  Port No.        No. of packets
----------------------------------------------------------------------
HTTPS                                               443       43208
HTTP                                                 80       11018
Dynamic Port                                      50930        2450
Registered Port                                   15000        2103
Registered Port                                    8160        1354


# 4. Total estimated traffic
Assume packet size is in bytes.

In [8]:
total_traffic = sum(list(df.packet_size))
total_traffic

4102693161

Converting to gigabytes:

In [9]:
totaltraf = total_traffic/(2**30)
print(f'{totaltraf:.2f} GB')

3.82 GB


# 5. Proportion of TCP and UDP Packets
TCP and UDP will have IP protocol values 6 and 17 respectively. Any other IP protocols will be put under "Other".

In [10]:
protocols = df['IP_protocol'].value_counts().to_frame()
protocols.columns = ['No. of packets']
protocols

,No. of packets
6,137707
17,36852
0,1216
50,718
47,664
1,60
58,10
41,3
2,2


Since highest non-TCP and UDP protocol value is 0, this will be the header value for "Other".

In [11]:
TCPdf = df.copy()
TCPdf.loc[(TCPdf["IP_protocol"] != 6 ) & (TCPdf["IP_protocol"]!=17), "IP_protocol"] = 0
protocols = TCPdf['IP_protocol'].value_counts().to_frame()
protocols.columns = ['No. of packets']
protocols

,No. of packets
6,137707
17,36852
0,2673


In [12]:
total_rows = df.shape[0]
protocolname = ("TCP", "UDP", "Other")
print(f'{"IP Protocol":15} {"Header Value":15} {"No. of packets":12} {"Percentage":9}')
print('-' * 70)
prodigit = 0
for index, row in protocols.iterrows():
    count = row["No. of packets"]
    percent = (count/total_rows)*100
    print(f'{protocolname[prodigit]:40}{index:15}{count:>12}{percent:>9.2f}%')
    prodigit+=1

IP Protocol     Header Value    No. of packets Percentage
----------------------------------------------------------------------
TCP                                                   6      137707    77.70%
UDP                                                  17       36852    20.79%
Other                                                 0        2673     1.51%


# 6. Top 5 Communication Pairs
For this we care only about the pairs, and not the destination or source addresses.


In [13]:
#Here we swap dst and src of a copy of df, and then concat them.
copydf = df.copy()
copydf.src_IP, copydf.dst_IP = np.where(True, [copydf.dst_IP, copydf.src_IP], [copydf.src_IP, copydf.dst_IP])
combined = pd.concat([df, copydf], ignore_index=True)
combined

,type,sflow_agent_address,inputPort,outputPort,src_MAC,dst_MAC,ethernet_type,in_vlan,out_vlan,src_IP,dst_IP,IP_protocol,ip_tos,ip_ttl,src_port,dst_port,tcp_flags,packet_size,IP_size,sampling_rate
0,FLOW,203.30.38.251,193,130,0031466b23cf,00239cd087c1,0x0800,919.0,919,74.125.10.59,137.132.38.238,17,0x00,63,443.0,62758,0x00,1396,1378,2048
1,FLOW,203.30.38.251,193,130,0031466b23cf,00239cd087c1,0x0800,919.0,919,173.194.22.215,137.132.228.29,6,0x00,63,443.0,39740,0x10,1438,1420,2048
2,FLOW,203.30.38.251,199,131,544b8cf9a7df,001cb0c88e40,0x0800,600.0,43,193.62.193.9,202.130.56.153,6,0x00,246,80.0,35794,0x18,272,250,2048
3,FLOW,203.30.38.251,193,131,0031466b23cf,001cb0c88e40,0x0800,919.0,43,74.125.130.132,192.122.131.33,6,0x00,52,443.0,5287,0x10,1418,1400,2048
4,FLOW,203.30.38.251,129,193,00135f21bc80,0031466b23cf,0x0800,11.0,919,155.69.160.69,74.125.68.139,17,0x60,59,48361.0,443,0x00,89,67,2048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354459,FLOW,203.30.38.251,193,131,0031466b23cf,001cb0c88e40,0x0800,919.0,43,192.122.131.146,74.125.10.23,6,0x00,63,443.0,50177,0x10,1490,1472,2048
354460,FLOW,203.30.38.251,200,199,80711fc76001,ccef48570144,0x0800,4095.0,601,210.48.222.13,13.107.4.50,6,0x00,112,80.0,50771,0x10,1448,1426,2048
354461,FLOW,203.30.38.251,200,131,80711fc76001,001cb0c88e40,0x0800,280.0,43,192.122.131.36,171.67.77.19,6,0x00,56,80.0,10452,0x10,1522,1500,2048
354462,FLOW,203.30.38.251,199,129,544b8cf9a7df,00135f21bc80,0x0800,600.0,11,155.69.59.240,133.11.245.204,17,0x00,55,50553.0,4500,0x10,150,128,2048


In [14]:
communications = combined.groupby(['src_IP', 'dst_IP']).size().sort_values(ascending=False).to_frame()
communications.columns = ['No. of packets']
communications

No. of packets
src_IP                                  dst_IP                                                 
103.37.198.100                          130.14.250.7                                       4201
130.14.250.7                            103.37.198.100                                     4201
192.122.131.36                          171.67.77.19                                       3628
171.67.77.19                            192.122.131.36                                     3628
129.99.230.54                           137.132.22.74                                      2417
...                                                                                         ...
155.69.5.81                             132.195.64.6                                          1
155.69.5.52                             202.130.97.175                                        1
                                        202.130.97.165                                        1
155.69.5.227                            193.54.107.40                                         1
ff02:0000:0000:0000:0000:0001:fffc:0005 fe80:0000:0000:0000:5257:a8ff:fe34:5f00               1

[39166 rows x 1 columns]

In [15]:
communications.drop(communications.index[1::2],0,inplace=True)
communications

No. of packets
src_IP         dst_IP                        
103.37.198.100 130.14.250.7              4201
192.122.131.36 171.67.77.19              3628
129.99.230.54  137.132.22.74             2417
137.131.17.212 137.132.228.42            2370
104.146.199.27 202.21.159.246            1794
...                                       ...
155.69.52.110  104.232.27.116               1
155.69.5.81    190.122.236.36               1
               132.68.225.14                1
155.69.5.52    202.130.97.175               1
155.69.5.227   193.54.107.40                1

[19583 rows x 1 columns]

In [16]:
communications = communications.head(5)

In [17]:
print(f'{"IP Address 1":30} {"IP Address 2":30} {"No. of Packets":15}')
print('-' * 80)
for index, row in communications.iterrows():
    print(f'{index[0]:30}{index[1]:30}{row["No. of packets"]:>15}')

IP Address 1                   IP Address 2                   No. of Packets 
--------------------------------------------------------------------------------
103.37.198.100                130.14.250.7                             4201
192.122.131.36                171.67.77.19                             3628
129.99.230.54                 137.132.22.74                            2417
137.131.17.212                137.132.228.42                           2370
104.146.199.27                202.21.159.246                           1794
